In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- James Tsai:
- Josh Ruiz:
- Danny Murphy:

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.121435315436 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

1. There are 11,314 instances in the dataset.

2. Each instance represents an article or document.

3. There are 20 classes, each a different newsgroup pertaining to a different document topic.

4. Time doesn't seem to be as important as topic in this dataset. The model is to be trained on documents within these 20 topic classes. If a document written in the past week fits roughly into one of these topics then we would expect the model to classify it well.

5. The data is represented as a sparse matrix. They are using the "bag-of-words" model where the 130,000 features represent all of the possible words, which means that many attributes will remain empty. As we can see in this calculation only .12 percent of the data are non-zero values.





___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [3]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Cosine'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:',euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:',cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a>0,b>0), 'ac:', jaccard(a>0,c>0), 'bc:',jaccard(b>0,c>0))

print('\n\nThe most appropriate distance is...' )
print(p)

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.09851846719 ac: 1.18914054254 bc: 0.917779422666
Cosine Distance
 ab: 0.603371411376 ac: 0.707027614956 bc: 0.421159534335
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.882113821138 ac: 0.875471698113 bc: 0.908794788274


The most appropriate distance is...
Cosine


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [4]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
K = [1, 2, 3, 4, 5]

for i in K:
    clf = KNeighborsClassifier(n_neighbors=i, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    acc = np.mean(clf.predict(X_test) == y_test)
    print('The accuracy of K = ', i, 'is', acc)
    
# acc = 0.2 # overwrite this with the actual accuracy

#=====================================

# print('Accuracy of classifier with %d neighbors is: %.2f'%(K,acc))


The accuracy of K =  1 is 0.596075658476
The accuracy of K =  2 is 0.545518826233
The accuracy of K =  3 is 0.53226091568
The accuracy of K =  4 is 0.522184903659
The accuracy of K =  5 is 0.506805727417


/Users/james/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


###### The best accuracy is 60% with 1 neighbor.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

Sparse data is mostly empty, so this makes it difficult to build an effective KDTree. The tree ends up being too deep with branches that aren't meaningful classifiers and therefore shouldn't be used.
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [5]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)

p = 'cosine'

param = ['euclidean', 'cosine', 'manhattan']

for i in param:
    clf = NearestCentroid(metric= i)
    clf.fit(X_train, y_train)
    acc = np.mean(clf.predict(X_test) == y_test)
    print('The accuracy of the parameter method = ', i, 'is', acc)


# fill in your code here


print('The best distance metric is: ', p)

The accuracy of the parameter method =  euclidean is 0.414530669966
The accuracy of the parameter method =  cosine is 0.488774969065
The accuracy of the parameter method =  manhattan is 0.180307583525
The best distance metric is:  cosine


/Users/james/anaconda/lib/python2.7/site-packages/sklearn/neighbors/nearest_centroid.py:138: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

Conditional probabilities is equal to the number of classes times the number of features. The probability of attribute i given class j is 1 / (130,000 * 20) = 1/2,600,000. So 2.6 million conditional probabilities need to be parameterized in our dataset. The prior is the probability of class j of which there are 20 options. So the number of priors is 20.


# Use this space for any calculations you might want to do



___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

1. GaussianNB assumes normally distributed continuous data because continuous data can be easily represented in a Gaussian distribution given the central limit theorem.

2. Yes it changes our answer. We know that text vector data is often stored in a sparse matrix and Multinomial Naive Bayes assumes that the data contains discrete features typically represented as text vectors making it a better choice than Gaussian. 

___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

alph = [0, 0.0001, 0.125, 0.25, 0.375, 0.5, 0.75, 1, 1.5, 2]

for i in alph:
    clf_mnb = MultinomialNB(alpha=i)
    clf_mnb.fit(X_train, y_train)
    acc = np.mean(clf_mnb.predict(X_test) == y_test)
    print('The accuracy of the alpha level (Multi) = ', i, 'is', acc)

    clf_bnb = BernoulliNB(alpha=i, binarize=0.0)
    clf_bnb.fit(X_train, y_train)
    acc = np.mean(clf_bnb.predict(X_test) == y_test)
    print('The accuracy of the alpha level (Bern) = ', i, 'is', acc)

# fill in your code here

            
print('These classifiers are so fast because... This is a sparse matrix containing mostly empty attributes.',
     'Naive Bayes algorithms are based on dot product calculations to classify a single parameter. This is easy to',
      'do on sparse data. The KNN and KDTree methods involve so many distance calculations that they become more',
      'computationally expensive.')
print('The alpha values control... The amount of smoothing in the model. A the alpha approaches zero the accuracy',
     'of our model increases. Were not 100% sure why, but assume that this has something to do with the large amount',
     'non-zero data in the dataset.')


/Users/james/anaconda/lib/python2.7/site-packages/sklearn/naive_bayes.py:664: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc)
/Users/james/anaconda/lib/python2.7/site-packages/sklearn/naive_bayes.py:766: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc)
/Users/james/anaconda/lib/python2.7/site-packages/sklearn/naive_bayes.py:785: RuntimeWarning: divide by zero encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/Users/james/anaconda/lib/python2.7/site-packages/sklearn/naive_bayes.py:788: RuntimeWarning: invalid value encountered in add
  jll += self.class_log_prior_ + neg_prob.sum(axis=1)


The accuracy of the alpha level (Multi) =  0 is 0.105532968004
The accuracy of the alpha level (Bern) =  0 is 0.0424253137705
The accuracy of the alpha level (Multi) =  0.0001 is 0.884921336397
The accuracy of the alpha level (Bern) =  0.0001 is 0.8458546933
The accuracy of the alpha level (Multi) =  0.125 is 0.827293618526
The accuracy of the alpha level (Bern) =  0.125 is 0.789110836132
The accuracy of the alpha level (Multi) =  0.25 is 0.802191974545
The accuracy of the alpha level (Bern) =  0.25 is 0.754640268694
The accuracy of the alpha level (Multi) =  0.375 is 0.779918684815
The accuracy of the alpha level (Bern) =  0.375 is 0.727594131165
The accuracy of the alpha level (Multi) =  0.5 is 0.76365564787
The accuracy of the alpha level (Bern) =  0.5 is 0.702315715043
The accuracy of the alpha level (Multi) =  0.75 is 0.734134700371
The accuracy of the alpha level (Bern) =  0.75 is 0.660597489836
The accuracy of the alpha level (Multi) =  1 is 0.705674385717
The accuracy of the al

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.